In [137]:
import pandas as pd
import numpy as np
#워닝 메시지
import warnings
warnings.filterwarnings('ignore')


In [138]:
train = pd.read_csv('./Dataset/Undersampling/OSS_0.33_train.csv', encoding='euc-kr')
test = pd.read_csv('./Dataset/test_다시.csv', encoding='euc-kr')

In [139]:
X_train_sum=train[['유동자산회전률', '총자산대비잉여현금흐름','자기자본구성비율', 'log자산총계','자기자본회전률', '순운전자본회전률', '자기자본증가율', '총자본증가율', '총자산대비현금흐름', '총자본투자효율']]
y_train = train[['t-1감사의견코드']]
X_test_sum=test[['유동자산회전률', '총자산대비잉여현금흐름','자기자본구성비율', 'log자산총계','자기자본회전률', '순운전자본회전률', '자기자본증가율', '총자본증가율', '총자산대비현금흐름', '총자본투자효율']]
y_test = test[['t-1감사의견코드']]

In [140]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [141]:
# ------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import xgboost as xgb
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC


def perform_model_grid_search(model_class, X_train, y_train, param_grid, k_fold=5):
    # Stratified k-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # 모델 초기화
    model = model_class()

    # 그리드 서치 설정
    grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=cv, verbose=0, n_jobs=-1)

    # 모델 학습 및 튜닝
    grid_search.fit(X_train, y_train)

    # 최적 하이퍼파라미터 출력
    print("Best Hyperparameters:", grid_search.best_params_)

    # 평균 평가 지표 계산
    mean_f1_score = np.mean(grid_search.cv_results_['mean_test_score'])
    print("Mean F1 Score:", mean_f1_score)

    return grid_search.best_params_, mean_f1_score

def evaluate_model_with_best_params_and_threshold(model_class, X_train, y_train, X_test, y_test, best_params, threshold=0.63, k_fold=5):
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    model = model_class(**best_params, verbose=0)

    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        model.fit(X_train_fold, y_train_fold)

        y_pred_fold_prob = model.predict_proba(X_test_fold)[:, 1]
        y_pred_fold = (y_pred_fold_prob > threshold).astype(int)

        accuracy = accuracy_score(y_test_fold, y_pred_fold)
        precision = precision_score(y_test_fold, y_pred_fold)
        recall = recall_score(y_test_fold, y_pred_fold)
        f1 = f1_score(y_test_fold, y_pred_fold)
        conf_matrix = confusion_matrix(y_test_fold, y_pred_fold)

        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    y_pred_final_prob = best_model.predict_proba(X_test)[:, 1]
    y_pred_final = (y_pred_final_prob > threshold).astype(int)

    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results with Threshold =", threshold)
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list

# --------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------
# 모델 선택 및 하이퍼파라미터 그리드 설정
model_choices = {
    "LogisticRegression": (LogisticRegression, {
        'C': [0.01, 0.1, 1, 10],
        'penalty': ['l1', 'l2']
    }),
    "RandomForest": (RandomForestClassifier, {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt']
    }),
    "XGBoost": (xgb.XGBClassifier, {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'n_estimators': [100, 200, 300]
    }),
    "LightGBM": (LGBMClassifier, {
        'max_depth': [11,12,13],
        'learning_rate': [0.057,0.055],
        'n_estimators': [120,110]
    }),
    "CatBoost": (CatBoostClassifier, {
        'depth': [10,12,14],
        'learning_rate': [0.09, 0.1,0.12],
        'iterations': [120,140,160]
    }),
    "SVC": (SVC, {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto']
    })
}


In [142]:
# 모델 선택
selected_model = "LightGBM"

# 모델 선택 및 설정
model_class, param_grid = model_choices[selected_model]

# 모델 하이퍼파라미터 튜닝
best_params, mean_f1_score = perform_model_grid_search(model_class, X_train_sum, y_train, param_grid)

# 모델 평가
evaluate_model_with_best_params_and_threshold(model_class, X_train_sum, y_train, X_test_sum, y_test, best_params)

# ----------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------

Best Hyperparameters: {'learning_rate': 0.057, 'max_depth': 12, 'n_estimators': 120}
Mean F1 Score: 0.7816694123333541
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000886 seconds.
You can set `force_col_wise=true` to remove the overhead.
Fold 1
Accuracy: 0.8906976744186047
Precision: 0.8381612090680101
Recall: 0.6932291666666667
F1 score: 0.7588369441277081
Confusion Matrix:
[[5563  257]
 [ 589 1331]]
------------------------------
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31)

([0.8906976744186047,
  0.8910852713178294,
  0.8874677002583979,
  0.8921049231166818,
  0.8884868846104148],
 [0.8381612090680101,
  0.8323057953144266,
  0.8314393939393939,
  0.8281911675741077,
  0.8268398268398268],
 [0.6932291666666667,
  0.7027589796980739,
  0.6855804268610098,
  0.7130208333333333,
  0.6963541666666667],
 [0.7588369441277081,
  0.7620660457239626,
  0.7514978601997147,
  0.7663028267562273,
  0.7560079163132599])